# Particle Trapping with Muon decay

Notes:
- Simulation is a three stage deal:
    - Generate muons isotropically within the DS trap region, allow them to propigate for 700 ns (no decay)
    - Use that output to further evolve remaining muons, allowing them to decay to electrons.
    - Run full output through TrkAna, see if any tracks are reconstructed.
- Their are two copies of each 2nd stage muon in `ntpart`.
    - Looks like copy 1 has t==0, p==100, pstop=p at 700ns
    - The second muon has t=~700, p==100, parent_pstop = original p_stop
- No muons have an initial timestamp > 700 ns
- No electrons are born before 700 ns


### Some Findings

Number of trapped muons per init muon p:
* p=140: 33000*300 = ~9.9mil

Number of unique events in which an electron with p>100 makes it farther upstream than 11200mm:
* Muon init p=140: ? Events

In [ ]:
from mu2e import mu2e_ext_path
from mu2e.dataframeprod import g4root_to_df
from root_pandas import read_root
import pandas as pd
from mu2e.mu2eplots import mu2e_plot3d_ptrap
from mu2e.mu2eplots import mu2e_plot3d_ptrap_traj
from mu2e.mu2eplots import mu2e_plot3d_ptrap_anim
from skhep.visual import MplPlotter as splt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,8)

In [ ]:
with open('/Volumes/DataDump/Mu2E/jobs_stage2.txt', 'r') as job_file:
    print job_file.readline()

In [ ]:
import mmap
def get_line_number(file_path):  
    fp = open(file_path, "r+")
    buf = mmap.mmap(fp.fileno(), 0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines

# Bookkeeping for high-stats samples

In [ ]:
from root_pandas import to_root, read_root
import tqdm
jobs_list_tmp = [15613699, 15613700, 15613702, 15613704]
jobs_path = '/Volumes/DataDump/Mu2E/jobs_stage2.txt'
#for i in tqdm.tqdm_notebook(jobs_list_tmp):
with open(jobs_path, 'r') as job_file:
    for line in tqdm.tqdm_notebook(job_file, total=get_line_number(jobs_path)):
        i = int(line)
        input_root = '/Volumes/DataDump/Mu2E/stage2-140p-hack/outstage/{}/00/00000/nts.bpollack.iso_muons_GA05_stage2_vd.v0.all.root'.format(i)
        output_root = '/Volumes/DataDump/Mu2E/stage2-140p-hack/outstage/{}/00/00000/nts.bpollack.iso_muons_GA05_stage2_vd.v0.skim.root'.format(i)
        df_ntpart = read_root(input_root, 'readvd/ntpart', ignore='*vd')
        df_nttvd = read_root(input_root, 'readvd/nttvd')
        #df_ntvd = read_root(input_root, 'readvd/ntvd')
    
        df_ntpart.set_index(['evt', 'subrun'], inplace=True)
        
        df_nttvd.set_index(['evt', 'subrun'], inplace=True)
        #df_ntvd.set_index(['evt', 'subrun'], inplace=True)
    
        good_index = df_ntpart.query('pdg==11 and p>75').index
        df_ntpart = df_ntpart.ix[good_index]
        #df_ntvd = df_ntvd.ix[good_index]
        df_nttvd = df_nttvd.ix[good_index]
        #df_ntpart['job'] = i
        df_ntpart.eval('job = {}'.format(i), inplace=True)
        df_ntpart.eval('x = x+3904', inplace=True)
        df_ntpart.eval('xstop = xstop+3904', inplace=True)
        #df_ntvd['job'] = i
        #df_nttvd['job'] = i
        df_nttvd.eval('job = {}'.format(i), inplace=True)
        df_nttvd.eval('x = x+3904', inplace=True)
        df_nttvd.eval('p = sqrt(px**2+py**2+pz**2)', inplace=True)
    
        df_ntpart.reset_index(inplace=True)
        df_nttvd.reset_index(inplace=True)
        #df_ntvd.reset_index(inplace=True)
    
        #df_ntvd.to_root(output_root, key='ntvd', mode = 'w')
        df_nttvd.to_root(output_root, key='nttvd', mode = 'w')
        df_ntpart.to_root(output_root, key='ntpart', mode = 'a')
    #print df_nttvd.head()


In [ ]:
input_root = '/Volumes/DataDump/Mu2E/pion-st-full-Mau13-trkana/nts.bpollack.pions_Mau13_trkana.v656.combined_all.root'
df_trkana = read_root(input_root, 'TrkAna/trkana', ['evtinfo', 'dem', 'uem', 'dmm'])

In [ ]:
df_trkana['dem_d0'] = df_trkana['dem_d0'].astype(float)
df_trkana.dem_d0.dtype

In [ ]:
cut_string = 'dem_trkqual>0.4 and dem_d0>-80 and dem_d0<105 and 450<(dem_d0+2.0/dem_om)<680 and 0.57735<dem_td<1.0'

In [ ]:
df_trkana.query(cut_string+' and 103.85<dem_mom<105.1').shape
#df_trkana.query(cut_string).shape

In [ ]:
df_trkana.query('90<dem_mom and dem_pdg==11').shape

In [ ]:
df_trkana.columns

In [ ]:
df_trkana.rename(columns={'evtinfo_eventid':'evt', 'evtinfo_runid':'run', 'evtinfo_subrunid':'subrun'}, inplace=True)

In [ ]:
df_trkana.set_index(['run', 'subrun', 'evt'], inplace=True)

In [ ]:
pd.to_pickle(df_trkana.query(cut_string+' and 103.85<dem_mom<105.1').index, 'golden_index.pkl')

In [ ]:
df_trkana.loc[df_trkana.query(cut_string+' and 103.85<dem_mom<105.1').index]

In [ ]:
splt.hist(df_trkana.query(cut_string).dem_mom)
plt.title('Reconstructed Electron Momemtum')
plt.xlabel('p (MeV)')

In [ ]:
import re
from root_pandas import to_root, read_root
import tqdm
jobs_path = '/Volumes/DataDump/Mu2E/jobs_trkana.txt'
with open(jobs_path, 'r') as job_file:
    for line in tqdm.tqdm_notebook(job_file, total=get_line_number(jobs_path)):
        i = int(line)
        input_root = '/Volumes/DataDump/Mu2E/trkana-140p-hack/outstage/{}/00/00000/nts.bpollack.iso_muons_GA05_trkana.v0.all.root'.format(i)
        input_log = '/Volumes/DataDump/Mu2E/trkana-140p-hack/outstage/{}/00/00000/log.bpollack.trkana-140p-hack.v0.all.log'.format(i)
        output_root = '/Volumes/DataDump/Mu2E/trkana-140p-hack/outstage/{}/00/00000/nts.bpollack.iso_muons_GA05_trkana.v0.skim.root'.format(i)
        
        df_trkana = read_root(input_root, 'TrkAna/trkana', ['evtinfo', 'dem', 'uem', 'dmm'])
        
        with open(input_log) as log_file:
            m = re.search('(?<=stage2-140p-hack/outstage/)\w+', log_file.read())
        jn = int(m.group(0))
        
        df_trkana['evtinfo_job'] = jn
        
        df_trkana.to_root(output_root, key='trkana', mode = 'w')
        

In [ ]:
input_log = '/Volumes/DataDump/Mu2E/trkana-140p-hack/outstage/15669486/00/00000/log.bpollack.trkana-140p-hack.v0.all.log'

In [ ]:
import re
with open(input_log) as f:
    m = re.search('(?<=stage2-140p-hack/outstage/)\w+', f.read())
jn = m.group(0)

In [ ]:
print int(jn)

In [ ]:
m.group(0)

In [ ]:
print 'hello'